In [11]:
%run ../../../forth.py

p e f o r t h    v1.28
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

*** Start self-test 
*** Data stack should be empty ... pass
*** Rreturn stack should have less than 2 cells ... pass
*** // adds help to the last word ... pass
*** TIB lines after \ should be ignored ... pass
*** slice ... pass
reDef ---
*** case ... endcase ... pass
*** modules lists imported modules ... pass
*** cd mimic DOS/Linux cd command ... pass
*** debugger commands x@ x> >x .sx xdrop xdropall ... pass
*** xstack commands x@ x> >x .sx xdrop xdropall ... pass
*** /// add comment to the last word ... pass
*** immediate makes the LAST an immediate word ... pass
*** compyle source code to function ... pass
*** </pyV> based on </py> and on compyle ... pass
*** interpret-only marks the last word an interpret-only word ... pass
*** immediate marks the last word an immediate word ... pass
*** compile-only marks last word as a compile-only word

# Quickstart: Learn how to submit batch jobs with the Azure Machine Learning Python SDK

In this quickstart, you learn how to submit a batch training job using the Python SDK. In this example, we submit the job to the 'local' machine (the compute instance you are running this notebook on). However, you can use exactly the same method to submit the job to different compute targets (for example, AKS, Azure Machine Learning Compute Cluster, Synapse, etc) by changing a single line of code. A full list of support compute targets can be viewed [here](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target). 

This quickstart trains a simple logistic regression using the [MNIST](https://docs.microsoft.com/azure/open-datasets/dataset-mnist) dataset and [scikit-learn](http://scikit-learn.org) with Azure Machine Learning.  MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of 28x28 pixels, representing a number from 0 to 9. The goal is to create a multi-class classifier to identify the digit a given image represents. 

You will learn how to:

> * Download a dataset and look at the data
> * Train an image classification model by submitting a batch job to a compute resource
> * Use MLflow autologging to track model metrics and log the model artefact
> * Review training results, find and register the best model

### Connect to your workspace and create an experiment

You start with importing some libraries and creating an experiment to track the runs in your workspace. A workspace can have multiple experiments, and all the users that have access to the workspace can collaborate on them. 

In [1]:
from azureml.core import Workspace
from azureml.core import Experiment

In [3]:
# connect to your workspace
ws = Workspace.from_config(r"d:\hcchen\OneDrive\文件\Jupyter Notebooks\Azure\.azureml\config.json")

In [4]:
experiment_name = "get-started-with-jobsubmission-tutorial"
exp = Experiment(workspace=ws, name=experiment_name)

### The MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corresponding class, `MNIST` in this case, to retrieve the data in different ways.

Follow this [how-to](https://aka.ms/azureml/howto/createdatasets) if you want to learn more about Datasets and how to use them.

咱碰到這個問題： No module named 'azureml.opendatasets'。查 pip list | find "azureml" 還真的沒有。查 google [找到答案 on this tutorial repo's issue list.](https://github.com/Azure/MachineLearningNotebooks/issues/518)。結果這個 datasets 恐怕很大。。。 做吧！

    pip install azureml-opendatasets 
    
這個 pip install 要連坐兩次才會成功。第一次的 error message 又是 pywin32 !! 如下：

    Installing collected packages: pywin32, py4j, pyspark, azureml-opendatasets
    Attempting uninstall: pywin32
    Found existing installation: pywin32 303
    Uninstalling pywin32-303:
      Successfully uninstalled pywin32-303
    ERROR: Could not install packages due to an OSError: [WinError 5] 存取被拒。: 'C:\\Users\\hcche\\AppData\\Roaming\\Python\\Python38\\site-packages\\~-n32\\win32api.pyd'
    Check the permissions.


In [7]:
from azureml.opendatasets import MNIST
mnist_file_dataset = MNIST.get_file_dataset()

In [13]:
# 查看這個 dataset 
%f mnist_file_dataset -->
%f mnist_file_dataset dir -->

mnist_file_dataset --> FileDataset
{
  "source": [
    "https://azureopendatastorage.blob.core.windows.net/mnist/*.gz"
  ],
  "definition": [
    "GetFiles",
    "ExpressionAddColumn",
    "StrReplace",
    "ExpressionAddColumn",
    "DropColumns",
    "RenameColumns",
    "DropColumns"
  ]
} (<class 'azureml.data.file_dataset.FileDataset'>)
mnist_file_dataset dir --> ['File', 'Tabular', 'add_tags', 'as_cache', 'as_download', 'as_hdfs', 'as_mount', 'as_named_input', 'data_changed_time', 'description', 'download', 'file_metadata', 'filter', 'get_all', 'get_by_id', 'get_by_name', 'get_partition_key_values', 'hydrate', 'id', 'mount', 'name', 'partition_keys', 'random_split', 'register', 'remove_tags', 'skip', 'tags', 'take', 'take_sample', 'to_path', 'unregister_all_versions', 'update', 'version'] (<class 'list'>)


### Define the Environment
An Environment defines Python packages, environment variables, and Docker settings that are used in machine learning experiments. Here you will be using a curated environment that has already been made available through the workspace. 

Read [this article](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments) if you want to learn more about Environments and how to use them.

"Environment" 是個新字 to me. 我以為是前兩天在 AutoML studio 上玩過的 "Experiment" 非也。先照用就對了。。。

In [14]:
from azureml.core.environment import Environment

# use a curated environment that has already been built for you

env = Environment.get(workspace=ws, 
                      name="AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36", 
                      version=1)

In [15]:
%f env --> # 
%f env dir -->

env --> Environment(Name: AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36,
Version: 1) (<class 'azureml.core.environment.Environment'>)
env dir --> ['add_private_pip_wheel', 'build', 'build_local', 'clone', 'databricks', 'docker', 'environment_variables', 'from_conda_specification', 'from_docker_image', 'from_dockerfile', 'from_existing_conda_environment', 'from_pip_requirements', 'get', 'get_image_details', 'inferencing_stack_version', 'label', 'list', 'load_from_directory', 'name', 'python', 'r', 'register', 'save_to_directory', 'spark', 'version'] (<class 'list'>)


### Configure the training job

Create a [ScriptRunConfig](https://docs.microsoft.com/python/api/azureml-core/azureml.core.script_run_config.scriptrunconfig?view=azure-ml-py) object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on. Configure the ScriptRunConfig by specifying:

* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will point to local compute
* The training script name, train.py
* An environment that contains the libraries needed to run the script
* Arguments required from the training script. 

In this run we will be submitting to "local", which is the compute instance you are running this notebook. If you have another compute target (for example: AKS, Azure ML Compute Cluster, Azure Databricks, etc) then you just need to change the `compute_target` argument below. You can learn more about other compute targets [here](https://docs.microsoft.com/azure/machine-learning/how-to-set-up-training-targets). 

心得：這算是有必要之惡，就是用個 object 來設定東西。

In [16]:
from azureml.core import ScriptRunConfig

args = ["--data-folder", mnist_file_dataset.as_mount(), "--regularization", 0.5]

src = ScriptRunConfig(
    source_directory="src",
    script="train.py",
    arguments=args,
    compute_target="local",
    environment=env,
)

### Submit the job

Run the experiment by submitting the ScriptRunConfig object. After this there are many options for monitoring your run. Once submitted, you can either navigate to the experiment "get-started-with-jobsubmission-tutorial" in the left menu item __Experiments__ to monitor the run (上 AutoML studio 網頁去查看), or you can monitor the run inline as the `run.wait_for_completion(show_output=True)` will stream the logs of the run. You will see that the environment is built for you to ensure reproducibility - this adds a couple of minutes to the run time. On subsequent runs, the environment is re-used making the runtime shorter.

一 run 又失敗了，看不懂的 error message, 啥 conda? 咱 Desktop MinAn 有哇？：

    Activity Failed:
    {
        "error": {
            "code": "UserError",
            "message": "Unable to run conda package manager. 
                        AzureML uses conda to provision python\nenvironments from a dependency specification. 
                        To manage the python environment\nmanually instead, set userManagedDependencies to True 
                        in the python environment\nconfiguration. 
                        To use system managed python environments, install conda from:
                        https://conda.io/miniconda.html",
            "messageParameters": {},
            "details": []
        },
        "time": "2022-03-05T14:58:18.850729Z"
    }


In [17]:
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: get-started-with-jobsubmission-tutorial_1646492292_095b0179
Web View: https://ml.azure.com/runs/get-started-with-jobsubmission-tutorial_1646492292_095b0179?wsid=/subscriptions/c27e04e6-ca10-453d-9c80-1931b65eb245/resourcegroups/my_azure_AutoML_resource_group/workspaces/my_azure_AutoML_workspace&tid=de0795e0-d7c0-4eeb-b9bb-bc94d8980d3b

Streaming azureml-logs/60_control_log.txt

[2022-03-05T14:58:16.722324] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Running: ['cmd.exe', '/c', 'C:\\Users\\hcche\\AppData\\Local\\Temp\\azureml_runs\\get-started-with-jobsubmission-tutorial_1646492292_095b0179\\azureml-environment-setup/conda_env_checker.bat']
Starting the daemon thread to refresh tokens in background for process with pid = 3240
Materialized conda environment not found on target: C:\Users\hcche/.azureml/envs/azureml_5fe324eb2cc5d6ab8afd92822e495375


[2022-03-05T14:58:16.949852] Logging experiment preparation status in history service.


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Unable to run conda package manager. AzureML uses conda to provision python\nenvironments from a dependency specification. To manage the python environment\nmanually instead, set userManagedDependencies to True in the python environment\nconfiguration. To use system managed python environments, install conda from:\nhttps://conda.io/miniconda.html",
        "messageParameters": {},
        "details": []
    },
    "time": "2022-03-05T14:58:18.850729Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Unable to run conda package manager. AzureML uses conda to provision python\\nenvironments from a dependency specification. To manage the python environment\\nmanually instead, set userManagedDependencies to True in the python env

## Register model

The training script used the MLflow autologging feature and therefore the model was captured and stored on your behalf. Below we register the model into the Azure Machine Learning Model registry, which lets you keep track of all the models in your Azure Machine Learning workspace.

Models are identified by name and version. Each time you register a model with the same name as an existing one, the registry assumes that it's a new version. The version is incremented, and the new model is registered under the same name.

When you register the model, you can provide additional metadata tags and then use the tags when you search for models.

In [ ]:
# register model
model = run.register_model(
    model_name="sklearn_mnist", model_path="model/model.pkl"
)
print(model.name, model.id, model.version, sep="\t")

You will now be able to see the model in the regsitry by selecting __Models__ in the left-hand menu of the Azure Machine Learning Studio.

## Control Cost

If you want to control cost you can stop the compute instance this notebook is running on by clicking the "Stop compute" button next to the status dropdown in the menu above.

 ## Next Steps

In this quickstart, you have seen how to run jobs-based machine learning code in Azure Machine Learning. 

It is also possible to use automated machine learning in Azure Machine Learning service to find the best model in an automated fashion. To see how this works, we recommend that you follow the next quickstart in this series, [**Fraud Classification using Automated ML**](../quickstart-azureml-automl/quickstart-azureml-automl.ipynb). This quickstart is focused on AutoML using the Python SDK.

# The End

In [18]:
s = "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Unable to run conda package manager. AzureML uses conda to provision python\\nenvironments from a dependency specification. To manage the python environment\\nmanually instead, set userManagedDependencies to True in the python environment\\nconfiguration. To use system managed python environments, install conda from:\\nhttps://conda.io/miniconda.html\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"2022-03-05T14:58:18.850729Z\"\n}"
print(s)

Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Unable to run conda package manager. AzureML uses conda to provision python\nenvironments from a dependency specification. To manage the python environment\nmanually instead, set userManagedDependencies to True in the python environment\nconfiguration. To use system managed python environments, install conda from:\nhttps://conda.io/miniconda.html",
        "messageParameters": {},
        "details": []
    },
    "time": "2022-03-05T14:58:18.850729Z"
}
